In [21]:
import torch
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import tqdm
import matplotlib.pyplot as plt 
import os
from shutil import copy

In [16]:
main_path = r"C:\Users\polme\OneDrive - UAB\Escritorio\Universitat\3r Carrera\1r Semestre\Vision and Learning\DeepLearning\dataset"

In [17]:
import xml.etree.ElementTree as ET
annotation_df = pd.DataFrame(columns = ['FileName', 'xmin','ymin','xmax','ymax',
                                        'width','height'])

path_xml = main_path + "\\annotations"
for filename in os.listdir(path_xml):
    annotation_list = []
    tree = ET.parse(os.path.join(path_xml,filename))
    root = tree.getroot()
    annotation_list.append(root.find('filename').text)
    #print(root.find('filename').text)
    for cord in root.find('object').find('bndbox'):
        annotation_list.append(int(cord.text))
    annotation_list.append(int(root.find('size').find('width').text))
    annotation_list.append(int(root.find('size').find('height').text))
    annotation_df.loc[len(annotation_df)] = annotation_list
    
annotation_df['center_x'] = (annotation_df['xmax'] + annotation_df['xmin'])/(2*annotation_df['width'])
annotation_df['center_y'] = (annotation_df['ymax'] + annotation_df['ymin'])/(2*annotation_df['height'])

annotation_df['bb_width'] = (annotation_df['xmax'] - annotation_df['xmin'])/annotation_df['width']
annotation_df['bb_height'] = (annotation_df['ymax'] - annotation_df['ymin'])/annotation_df['height']

annotation_df

,FileName,xmin,ymin,xmax,ymax,width,height,center_x,center_y,bb_width,bb_height
0,Cars0.png,226,125,419,173,500,268,0.645000,0.555970,0.386000,0.179104
1,Cars1.png,134,128,262,160,400,248,0.495000,0.580645,0.320000,0.129032
2,Cars10.png,140,5,303,148,400,225,0.553750,0.340000,0.407500,0.635556
3,Cars100.png,175,114,214,131,400,267,0.486250,0.458801,0.097500,0.063670
4,Cars101.png,167,202,240,220,400,300,0.508750,0.703333,0.182500,0.060000
...,...,...,...,...,...,...,...,...,...,...,...
428,Cars95.png,278,182,407,263,600,400,0.570833,0.556250,0.215000,0.202500
429,Cars96.png,133,126,261,160,400,248,0.492500,0.576613,0.320000,0.137097
430,Cars97.png,98,102,297,191,400,300,0.493750,0.488333,0.497500,0.296667
431,Cars98.png,85,196,247,259,370,400,0.448649,0.568750,0.437838,0.157500


In [18]:
df_train, df_test_val = train_test_split(annotation_df, test_size=0.25)
df_test, df_val = train_test_split(df_test_val, test_size=0.50)
[len(df_train), len(df_test), len(df_val)]

[324, 54, 55]

In [29]:
train_folder = './yolo/data_images/train'
values = df_train[['FileName','center_x','center_y','bb_width','bb_height']].values
for fname, x, y, w, h in values:
    label_txt = f'0 {x} {y} {w} {h}'
    copy(os.path.join(main_path + "/images",fname),
         os.path.join(train_folder,fname))
    fname_ = fname.split('.')[0]
    label_path = os.path.join(train_folder,fname_+'.txt')
    with open(label_path, mode='w') as f:
        f.write(label_txt)
        f.close()

In [30]:
test_folder = './yolo/data_images/test'
values = df_test[['FileName','center_x','center_y','bb_width','bb_height']].values
for fname, x, y, w, h in values:
    label_txt = f'0 {x} {y} {w} {h}'
    copy(os.path.join(main_path + "\\images",fname),
         os.path.join(test_folder,fname))
    fname_ = fname.split('.')[0]
    label_path = os.path.join(test_folder,fname_+'.txt')
    with open(label_path, mode='w') as f:
        f.write(label_txt)
        f.close()